In [34]:
import pandas as pd

Funtions

In [2]:
def SeperateDrugs_FailingReasons(input_df, substring, specific):
    selected_rows_substring = input_df[input_df['Reason for failing'].str.contains('|'.join(substring))]
    selected_rows_specific = input_df[input_df['Reason for failing'].isin(specific)]
    not_harmful_drugs = pd.concat([selected_rows_substring, selected_rows_specific])
    PM_ToxDrugs = pd.concat([input_df, not_harmful_drugs]).drop_duplicates(keep=False)
    return not_harmful_drugs, PM_ToxDrugs

def Standardize_dataset_format(df, Compound_name, Status, id, SMILES, Dataset):
    df_rename = pd.DataFrame()
    df_rename = df.rename(columns={Compound_name:'Name', id:'chembl_id', Status:'status', SMILES:'SMILES', Dataset: 'Dataset'})
    df_simplified = df_rename[['Name', 'status', 'SMILES','chembl_id', 'Dataset']]
    print("number of the simplified set: {}".format(len(df_simplified)))
    df_simplified_drop_duplicate = df_simplified.drop_duplicates(keep='first')
    print("number of the simplified set after dropping duplicate: {}".format(len(df_simplified_drop_duplicate)))
    df_simplified_drop_duplicateAndnan = df_simplified_drop_duplicate.dropna(subset=['SMILES'])
    print("number of the simplified set after dropping nan in SMILES: {}".format(len(df_simplified_drop_duplicateAndnan)))
    return df_simplified_drop_duplicateAndnan

def add_identifier_for_each_dataset(ori_dataFrame, DS_abbreviation):
    start = 1
    ori_dataFrame['DS_id'] = [DS_abbreviation + str(i) for i in range(start, start + len(ori_dataFrame))]
    return ori_dataFrame

## 1. literature: Post-marketing withdrawal of 462 medicinal products because of adverse drug reactions: a systematic review of the world literature 

In [ ]:
PostMarketing = pd.read_csv('../datasets/Post_Marketing_raw.csv')
print(len(PostMarketing))

462


Remove markers from the literture

In [20]:
PostMarketing['Compound'] = PostMarketing['Compound'].str.rstrip('‡')

Standardize the format of dataframe, including the source of the dataset, status and phase of the drugs, reason of failing

In [21]:
PostMarketing['Dataset'] = 'PostMarketing'
PostMarketing['status'] = 'Withdrawn'
PostMarketing['phase'] = 'marketed'
PostMarketing.rename(columns = {'Reason for withdrawal':'Reason for failing'}, inplace = True)

add identifier of each dataset, simplify the dataframe and uniform the expression of failing reason

In [22]:
PostMarketing = add_identifier_for_each_dataset(PostMarketing, 'PM-')
PostMarketing = PostMarketing[['Dataset', 'Compound', 'status', 'Reason for failing', 'phase']]
PostMarketing['Reason for failing'] = PostMarketing['Reason for failing'].str.replace('(L|l)iver','hepatotoxicity')

/home/hzhang/anaconda3/envs/hzhang/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


Need to manually check the not harmful drugs

In [ ]:
substring = ['effective', '(I|i)nsufficient', 'efficacy', 'HIV', 'regulation', 'alternative','Not',
            '(I|i)nteraction','(I|i)nappropriate', '(A|a)buse', 'Resistance', 'potential', 'Negative']
specific = ['(O|o)verdose', 'Drug dependence', 'Drug interactions', 'Drug abuse', 'Superseded',
            'infection', 'Negative', 'Potential', 'Suspending', 'Drug dependence, drug overdose', 'Suicides',
            'Risk of overdose', 'Drug overdose']

In [ ]:
not_harmful_drugs, PM_ToxDrugs = SeperateDrugs_FailingReasons(PostMarketing, substring, specific)

## 2. literature: FDA-approved drug labeling for the study of drug-induced liver injury (DILI) (LTKB)

In [ ]:
DILI = pd.read_csv("../datasets/DILI_raw.csv")
print(len(DILI))

137


In [28]:
DILI.rename(columns = {'Generic name':'Compound', 'Status/\n label':'status', 'CAS#':'CAS'}, inplace = True)
DILI['Dataset'] = 'DILI'
DILI['phase'] = 'marketed'
DILI['Reason for failing'] = 'hepatotoxicity'

vals_to_replace = {'WD':'Withdrawn', 'D':'Terminated', 'BW':'Black Box Warning', 'WP':'Warnings and Precautions'}
DILI['status'] = DILI['status'].map(vals_to_replace)
DILI = DILI[['Dataset', 'Compound', 'status', 'Reason for failing', 'phase', 'Therapeutic category', 'Brand name', 'DILI\n severity\n score', 'CAS', 'Vendor']]
print(len(DILI))

137


In [29]:
DILI_withdrawn = DILI.loc[DILI['status'] == 'Withdrawn']
print(len(DILI_withdrawn))

54


## 3. ChEMBL Drug warning_withdrawn

In [ ]:
Chembl_withdrawn = pd.read_csv('../datasets/ChEMBL_withdrawn/Drug_warnings_withdrawn.csv', sep=';')
Chembl_chemreps = pd.read_csv('../datasets/ChEMBL_withdrawn/chembl_33_chemreps.txt', sep='\t')

In [ ]:
Chembl_withdrawn['Dataset'] = 'Chembl_withdrawn'
Chembl_withdrawn.rename(columns = {'Parent Molecule ChEMBL ID':'chembl_id'}, inplace=True)

In [18]:
Merge_SMILES_chembl_withdrawn = Chembl_withdrawn.merge(Chembl_chemreps[['chembl_id', 'canonical_smiles', 'standard_inchi']], how='left', on='chembl_id')
print(len(Merge_SMILES_chembl_withdrawn))

308


In [ ]:
Merge_SMILES_chembl_withdrawn[Merge_SMILES_chembl_withdrawn.canonical_smiles.isna()]

Manually check the data and add the uncompleted SMILES

In [ ]:
Merge_SMILES_chembl_withdrawn = pd.read_csv('../datasets/ChEMBL_withdrawn/Merge_SMILES_chembl_withdrawn_ManuallyCheck.csv')

In [ ]:
Merge_SMILES_chembl_withdrawn_dropna = Merge_SMILES_chembl_withdrawn.dropna(subset=['canonical_smiles'])

294


In [ ]:
mask = Merge_SMILES_chembl_withdrawn_dropna['Warning Class'].str.contains('|'.join(['misuse', 'None']))
Merge_SMILES_chembl_withdrawn_dropna = Merge_SMILES_chembl_withdrawn_dropna[~mask]

## 4. DrugBank_approved 

In [ ]:
drugbank_approved = pd.read_csv('../datasets/approved/DrugBank/drugbank_approved.csv')
drugbank_approved['Dataset'] = 'drugbank_approved'
print(len(drugbank_approved))
drugbank_approved.columns

2735


Index(['DrugBank ID', 'Name', 'CAS Number', 'Drug Groups', 'InChIKey', 'InChI',
       'SMILES', 'Formula', 'KEGG Compound ID', 'KEGG Drug ID',
       'PubChem Compound ID', 'PubChem Substance ID', 'ChEBI ID', 'ChEMBL ID',
       'HET ID', 'ChemSpider ID', 'BindingDB ID', 'Dataset'],
      dtype='object')

Check whether there is BiotechDrug in DrugBank_approved set

In [ ]:
drugbank_approved_extra = pd.read_csv('../datasets/approved/DrugBank/drugbank_approved_extra.csv')
print(len(drugbank_approved_extra))
drugbank_approved_extra.columns

4304


Index(['DrugBank ID', 'Name', 'CAS Number', 'Drug Type', 'KEGG Compound ID',
       'KEGG Drug ID', 'PubChem Compound ID', 'PubChem Substance ID',
       'ChEBI ID', 'PharmGKB ID', 'HET ID', 'UniProt ID', 'UniProt Title',
       'GenBank ID', 'DPD ID', 'RxList Link', 'Pdrhealth Link', 'Wikipedia ID',
       'Drugs.com Link', 'NDC ID', 'ChemSpider ID', 'BindingDB ID', 'TTD ID'],
      dtype='object')

In [ ]:
drugbank_approved_addInfo_onDrugbankID = drugbank_approved.merge(drugbank_approved_extra[['DrugBank ID', 'CAS Number', 'Drug Type']], how='left', on='DrugBank ID')
print(len(drugbank_approved))
print(len(drugbank_approved_addInfo_onDrugbankID))

2735
2735


In [20]:
drugbank_approved_addInfo_onDrugbankID['Drug Type'].unique()

array(['SmallMoleculeDrug'], dtype=object)

In [25]:
drugbank_approved_OnlyKeepApproved = drugbank_approved.loc[drugbank_approved['Drug Groups'] == 'approved']
print(len(drugbank_approved_OnlyKeepApproved))

1095


Manually check drugbank_approved set

In [26]:
check_nan = drugbank_approved_OnlyKeepApproved[['Name', 'CAS Number', 'SMILES', 'ChEMBL ID']]
check_nan = check_nan[check_nan[['SMILES', 'ChEMBL ID']].isna().any(axis=1)]
print(len(check_nan))

168


In [ ]:
check_nan = pd.read_csv('../datasets/approved/DrugBank/check_nan.csv', sep='\t')
drugbank_approved_OnlyKeepApproved_dropna = drugbank_approved_OnlyKeepApproved.dropna(subset=['ChEMBL ID', 'SMILES'])
print(len(drugbank_approved_OnlyKeepApproved_dropna))

927


## 5. Withdrawn 2.0 -- Manually curated

In [ ]:
withdrawn_manual_processed = pd.read_csv("../datasets/ToxicAndBengin_pool/withdrawn2_ManualProcessed.csv")
withdrawn_manual_processed['status'] = 'withdrawn'
withdrawn_manual_processed['dataset'] = 'Withdrawn2.0'
withdrawn_manual_processed['ChEMBL ID'] = 'ChEMBL ID'
withdrawn_GoTo_Toxic = withdrawn_manual_processed.loc[withdrawn_manual_processed['comment [Toxic(T)/Benign(B)/hold-out(-)]'] == 'T']
withdrawn_GoTo_Benign = withdrawn_manual_processed.loc[withdrawn_manual_processed['comment [Toxic(T)/Benign(B)/hold-out(-)]'] == 'B']

In [32]:
print(len(withdrawn_GoTo_Toxic))
print(len(withdrawn_GoTo_Benign))

10
37


## 6. Standardization of the data sets

benign (after name matching)

In [ ]:
benign_set = pd.read_csv("../datasets/data_aact/Pipe-Delimited/check_other_phase_1/bad,good,multi_drugs/good_part_process/final_set/result_total(1-5).csv", sep='|')
benign_ManuallyCurated_set = pd.read_csv("../datasets/data_aact/Pipe-Delimited/check_other_phase_1/bad,good,multi_drugs/good_part_process/final_set/df2_goodpart_ManuallyCurated.csv", sep=',')
drugbank_approved = pd.read_csv("../datasets/approved/DrugBank/drugbank_approved_OnlyKeepApproved_dropna.csv", sep=',')

In [51]:
benign_set['Dataset'] = 'clinical_trial'
benign_ManuallyCurated_set['Dataset'] = 'clinical_trial'

In [52]:
print(len(benign_ManuallyCurated_set))
benign_ManuallyCurated_set_WithSmiles = benign_ManuallyCurated_set.loc[~benign_ManuallyCurated_set['SMILES'].isna()]
print(len(benign_ManuallyCurated_set_WithSmiles))

1269
298


In [ ]:
benign_ManuallyCurated_set_WithSmiles = benign_ManuallyCurated_set_WithSmiles.rename(columns={'Name':'Name_ori'})
benign_set_S = Standardize_dataset_format(benign_set, 'Name', 'phase', 'ChEMBL ID', 'SMILES', 'Dataset')
benign_ManuallyCurated_set_S = Standardize_dataset_format(benign_ManuallyCurated_set_WithSmiles, 'name_to_use', 'phase', 'ChEMBL ID', 'SMILES', 'Dataset')
drugbank_approved_S = Standardize_dataset_format(drugbank_approved, 'Name', 'Drug Groups', 'ChEMBL ID', 'SMILES', 'Dataset')
withdrawn_GoTo_Benign_S = Standardize_dataset_format(withdrawn_GoTo_Benign, 'Name', 'status', 'ChEMBL ID', 'preprocessedSmiles', 'dataset')

Toxic

In [ ]:
PM_ToxDrugs[['ChEMBL ID', 'SMILES']] = ' '
DILI_withdrawn[['ChEMBL ID', 'SMILES']] = ' '

In [ ]:
Drugbank_approved_simplified = Standardize_dataset_format(drugbank_approved_OnlyKeepApproved_dropna, 'Name', 'Drug Groups', 'ChEMBL ID', 'SMILES', 'Dataset')
Chembl_withdrawn_simplified = Standardize_dataset_format(Merge_SMILES_chembl_withdrawn_dropna, 'Parent Molecule Name', 'Warning Type', 'chembl_id', 'canonical_smiles', 'Dataset')
DILI_simplified = Standardize_dataset_format(DILI_withdrawn, 'Compound', 'status', 'ChEMBL ID', 'SMILES', 'Dataset')
PostMarketing_simplified = Standardize_dataset_format(PM_ToxDrugs, 'Name', 'status', 'ChEMBL ID', 'SMILES', 'Dataset')

Concatenate withdrawn data sets and drop duplicates

In [ ]:
list_withdrawn = [PostMarketing_simplified, Chembl_withdrawn_simplified, DILI_simplified]
combine_withdrawn = pd.concat(list_withdrawn)

In [ ]:
withdrawn_GoTo_Toxic_S = Standardize_dataset_format(withdrawn_GoTo_Toxic, 'Name', 'status', 'ChEMBL ID', 'preprocessedSmiles', 'dataset')

In [ ]:
combine_withdrawn = pd.read_csv('../datasets/toxic_pool_recheck/withdrawn.csv')
combine_withdrawn.Name

0               Acetanilide
1                 Acetarsol
2         Acetylfuratrizine
3                 Acitretin
4      Acridine derivatives
               ...         
598            troglitazone
599           trovafloxacin
600           xenazoic acid
601            ximelagatran
602               zimeldine
Name: Name, Length: 603, dtype: object